In [5]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,545 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,01

In [6]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-22 00:29:50--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.67MB/s    in 0.5s    

2023-03-22 00:29:51 (1.67 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [7]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [12]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url_home_imp = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"
spark.sparkContext.addFile(url_home_imp)
home_imp_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
home_imp_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

In [13]:
# Get the number of rows in the DataFrame.
rows_home = home_imp_df.count()
rows_home


2634781

# Transform the Data

## Create the "review_id_table".

In [14]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = home_imp_df.select('review_id','customer_id','product_id','product_parent','review_date')
# pyspark.sql.functions.to_date('review_date', format='None')
review_id_df.select(to_date(review_id_df.review_date).alias('date')).collect()
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R215C9BDXTDQOW|   48881148|B00FR4YQYK|     381800308| 2015-08-31|
|R1DTPUV1J57YHA|   47882936|B00439MYYE|     921341748| 2015-08-31|
| RFAZK5EWKJWOU|   44435471|B00002N762|      56053291| 2015-08-31|
|R2XT8X0O0WS1AL|   28377689|B000QFCP1G|     595928517| 2015-08-31|
|R14GRNANKO2Y2J|   50134766|B00WRCRKOI|     417053744| 2015-08-31|
|R2BLF9VYL24LCQ|   14066511|B00NIH88EW|     275395071| 2015-08-31|
|R1GI9UW5KJ671O|   15211046|B005B9CI96|     856617815| 2015-08-31|
|R2H5CEJN863M86|   14862498|B008L0OMWI|     125102494| 2015-08-31|
| R5PPDHFOZ3SMU|   23617292|B00P9FTC6O|     523110842| 2015-08-31|
| RE1L9IENKJJ7Y|   35820485|B00K6BQEHQ|     797306964| 2015-08-31|
|R3CZ0990QC2Z0H|   47162350|B008BYQCWM|     865874404| 2015-08-31|
|R3UMMD2IO29QSP|   31884789|B003BLHTOU|     183592595| 2015-08

## Create the "products" Table

In [15]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns.
product_df = home_imp_df.select('product_id','product_title') 
product_df = product_df.dropDuplicates(['product_id','product_title'])
product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0136K7XN2|Brightown 72ft/22...|
|B013UDQE26|Amdirect Folding ...|
|B005WQIDHY|Woods 50007 Indoo...|
|B000H5QLJC|Dap 18013 Kwik-Se...|
|B002I0GRRA|KOHLER Cimarron C...|
|B00O4CUB7M|Kraus Nola Single...|
|B00857R9PY|Park Madison Ligh...|
|B009IJ2NBI|TOTO Washlet S350...|
|B00IZA2TAQ|XKTTSUEERCRR Wate...|
|B008UDP4PA|35265B Genie 90FT...|
|B004TSYJSI|Moen LR2356DBN 16...|
|B003YUGQWE|General Finishes ...|
|B0006VVN1I|Culligan Certifie...|
|B00CJ5EO2E|Gorilla Super Glu...|
|B00RBYV0JS|Decor Star TPC11 ...|
|B007VP7QCC|LiftMaster 1345 C...|
|B00106L9AS|Moonrays 91251 Co...|
|B011SXJCN8|Dean Sports Alumi...|
|B000R89WZO|Jaw Inserts for 4...|
|B000Z6G008|Westinghouse 6751...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [16]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = home_imp_df.select('customer_id')
customers_df = customers_df.groupBy('customer_id').count()
customers_df = customers_df.withColumnRenamed('count','customer_count')
customers_df.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28377689|             1|
|   28258386|             2|
|    9967574|             1|
|   25153155|             1|
|   10088068|             1|
|   45657423|             1|
|   19021463|             1|
|   41413793|             2|
|   43789873|             2|
|   12406466|             1|
|   41045019|             1|
|   35535911|             2|
|   47108763|             1|
|   48113150|             1|
|   18201417|             9|
|   26079415|             3|
|   36114891|             7|
|    3712628|             1|
|   37499901|             2|
|   47321438|             2|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [18]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = home_imp_df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R215C9BDXTDQOW|          4|            0|          0|   N|
|R1DTPUV1J57YHA|          5|            0|          0|   N|
| RFAZK5EWKJWOU|          5|            0|          0|   N|
|R2XT8X0O0WS1AL|          5|            0|          0|   N|
|R14GRNANKO2Y2J|          5|            0|          0|   N|
|R2BLF9VYL24LCQ|          5|            1|          1|   N|
|R1GI9UW5KJ671O|          5|            0|          0|   N|
|R2H5CEJN863M86|          5|            0|          1|   N|
| R5PPDHFOZ3SMU|          5|            0|          0|   N|
| RE1L9IENKJJ7Y|          1|            0|          0|   N|
|R3CZ0990QC2Z0H|          5|            0|          0|   N|
|R3UMMD2IO29QSP|          5|            0|          0|   N|
| R541LE5J3OJH2|          5|            0|          0|   N|
|R10KD9FIE6Y1AS|          5|            

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS


In [ ]:
# Write products_df to table in RDS


In [ ]:
# Write customers_df to table in RDS


In [ ]:
# Write vine_df to table in RDS
